In [2]:
import os

In [3]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [4]:
! pip install torch torchtext transformers sentencepiece pandas tqdm datasets accelerate bitsandbytes peft

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [5]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

/usr/local/lib/python3.10/dist-packages/numpy/lib/scimath.py:46: RuntimeWarning: divide by zero encountered in log
  _ln2 = nx.log(2.0)


In [6]:
# load data
data = load_dataset("csv", data_files="merged_file.csv")

In [7]:
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'cell_id_x', 'notebook_id', 'code_imports', 'defined_functions', 'source_x', 'cell_id_y', 'source_y'],
        num_rows: 828122
    })
})

In [8]:
updated_data = [{'Code': item['source_x'], 'Description': item['source_y']} for item in data["train"]]

In [9]:
df = pd.DataFrame(updated_data)

In [10]:
df.head(50)

,Code,Description
0,"import os\nPROJECT = ""cloud-training-demos"" # ...",# MNIST Image Classification with TensorFlow o...
1,%%bash\ngcloud config set project $PROJECT\ngc...,# MNIST Image Classification with TensorFlow o...
2,%%bash\nrm -rf mnistmodel.tar.gz mnist_trained...,## Run as a Python module\n\nIn the previous n...
3,%%bash\nOUTDIR=gs://${BUCKET}/mnist/trained_${...,"**Now, let's do it on Cloud ML Engine so we ca..."
4,from google.datalab.ml import TensorBoard\nTen...,## Monitoring training with TensorBoard\n\nUse...
5,"for pid in TensorBoard.list()[""pid""]:\n Ten...",## Monitoring training with TensorBoard\n\nUse...
6,"%%bash\nMODEL_NAME=""mnist""\nMODEL_VERSION=${MO...",## Deploying and predicting with model\n\nDepl...
7,"import json, codecs\nimport matplotlib.pyplot ...","To predict with the model, let's take one of t..."
8,%%bash\ngcloud ml-engine predict \\n --mode...,Send it to the prediction service
9,trainingInput:\n scaleTier: CUSTOM\n mas...,## DO NOT RUN anything beyond this point\n\nTh...


In [11]:
df['']

KeyError: ''

In [12]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from accelerate import Accelerator

In [13]:
#if torch.cuda.is_available():
#    device = torch.device('cuda')

#else:
#    try:
#        device = torch.device('mps')
#    except Exception:
#        device = torch.device('cpu')
accelerator = Accelerator()
device = accelerator.device

In [14]:
device

device(type='cuda')

In [15]:
model_name = "sagard21/python-code-explainer"

# Adding the bitsandbytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=bnb_config).to(device)

In [16]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32100, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [17]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [18]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [19]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "k", "v", "o", "up_proj", "down_proj", "gate_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 742358016 || trainable%: 0.6356221524251716


In [20]:
BATCH_SIZE = 9

In [21]:
df.describe()

,Code,Description
count,789629,826331
unique,552574,237424
top,df.head(),# Vertical Scan with Space Charge
freq,1028,1015


In [22]:
# Dataset preparation
class LanguageDataset(Dataset):
    def __init__(self, _df, _tokenizer, max_length=1024):
        self.labels = _df.columns
        self.data = _df.to_dict(orient='records')
        #print(self.data[:10])
        self.tokenizer = _tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][self.labels[0]]
        y = self.data[idx][self.labels[1]]
        text = f"{x} | {y}"
        print(text)
        tokens = self.tokenizer.encode_plus(text,
                                            return_tensors='pt',
                                            max_length=1024,
                                            truncation=True,
                                            padding='max_length')
        return tokens

In [23]:
data_sample = LanguageDataset(df, tokenizer)

In [24]:
for idx in range(10):
    print(data_sample.__getitem__(idx))

import os
PROJECT = "cloud-training-demos" # REPLACE WITH YOUR PROJECT ID
BUCKET = "cloud-training-demos-ml" # REPLACE WITH YOUR BUCKET NAME
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE = "dnn"  # "linear", "dnn", "dnn_dropout", or "cnn"

# Do not change these
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["MODEL_TYPE"] = MODEL_TYPE
os.environ["TFVERSION"] = "1.13"  # Tensorflow version | # MNIST Image Classification with TensorFlow on Cloud ML Engine

This notebook demonstrates how to implement different image models on MNIST using Estimator. 

Note the MODEL_TYPE; change it to try out different models
{'input_ids': tensor([[   1, 5666, 1140,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION | # MNIST Image Classification with TensorFlow on Cloud ML Engine

This notebook demonst

In [25]:
data_sample.__len__()

828122

In [26]:
train_size = int(0.8 * len(df))
val_size = len(df) - train_size

train_data, val_data = random_split(data_sample, [train_size, val_size])

In [27]:
len(train_data)

662497

In [28]:
len(val_data)

165625

In [29]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

In [30]:
num_epochs = 5

In [31]:
batch_size = BATCH_SIZE
model_name = 'sagard21/python-code'
gpu = 0

In [32]:
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [33]:
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu', 'training_loss', 'validation_loss', 'epoch_duration_sec'])

In [34]:
results

,epoch,transformer,batch_size,gpu,training_loss,validation_loss,epoch_duration_sec


In [35]:
len(train_data)

662497

In [36]:
len(val_data)

165625

In [37]:
len(train_data) + len(val_data)

828122

In [38]:
train_data[4]

neuron_id = np.random.randint(0, config.rnn_dim)

sentence_plot(some_test_sentences_contra_pad,
              news_contra[:, :, neuron_id],
              title="New candidate, neuron {} (only contradictory sentences)".format(neuron_id)) | - ### New candidate for hidden state


{'input_ids': tensor([[   1, 4644,  295,  ...,    2,    2,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}

In [39]:
val_data[3]

css_c = coord.SkyCoord(ra=css['RA']*u.degree, 
                       dec=css['Dec']*u.degree) | ## CSS


{'input_ids': tensor([[   1, 5212,   67,  ...,    2,    2,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}

In [40]:
for idx in range(20):
    print("Printing the", idx,"th row here:", train_data[idx],"\n")
    print("Also printing the input_ids:", train_data[idx]['input_ids'].numpy().size)

#descriptives of missing
vrs = ['retail','food_drugs','work','homes','pct_social_distancing']
for x in vrs:
    df_all['miss_{}'.format(x)] = np.where(df_all[x].isnull(),1,0)
    print('\nMissing Comparison for -{}\n{}'.format(x,df_all.drop_duplicates(['FIPS','miss_{}'.format(x)]).groupby(['miss_{}'.format(x)])['POP_ESTIMATE_2018'].describe())) | # Models - data transformation
Printing the 0 th row here: {'input_ids': tensor([[ 1,  7, 72,  ...,  2,  2,  2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])} 

#descriptives of missing
vrs = ['retail','food_drugs','work','homes','pct_social_distancing']
for x in vrs:
    df_all['miss_{}'.format(x)] = np.where(df_all[x].isnull(),1,0)
    print('\nMissing Comparison for -{}\n{}'.format(x,df_all.drop_duplicates(['FIPS','miss_{}'.format(x)]).groupby(['miss_{}'.format(x)])['POP_ESTIMATE_2018'].describe())) | # Models - data transformation
Also printing the input_ids: 1024
def logistic_regression(X, y, initializer=None, seed=42, learning_

In [41]:
from transformers import T5Model

class CustomT5Model(T5Model):
    def __init__(self, config):
        super().__init__(config)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None):
        # Custom logic for handling input_ids with length 1024
        if input_ids.size(1) > 1024:
            # Truncate or process input_ids as needed
            input_ids = input_ids[:, :1024]
        
        # Call the base T5Model's forward method with modified input_ids
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask)
        
        return outputs


In [42]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=2,
        max_steps=2000,
        learning_rate=5e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

trainer.train()

2024-04-03 12:04:40.694887: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 12:04:41.252607: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 12:04:41.253757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 12:04:41.341498: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 12:04:41.521426: I tensorflow/core/platform/cpu_feature_guar

from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test | Now that we've read the raw training and testing data from the downloaded dataset, we will combine the positive and negative reviews and shuffle the resulting records.
df.drop(['Name','Ticket'],axis=1,

ValueError: too many values to unpack (expected 2)

In [53]:
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_data:
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for val_inputs, val_targets in val_loader:
            val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)

            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_targets).item()

            _, predicted = torch.max(val_outputs, 1)
            correct_predictions += (predicted == val_targets).sum().item()
            total_samples += val_targets.size(0)

        val_accuracy = correct_predictions / total_samples
        average_val_loss = val_loss / len(val_loader)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {average_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

#descriptives of missing
vrs = ['retail','food_drugs','work','homes','pct_social_distancing']
for x in vrs:
    df_all['miss_{}'.format(x)] = np.where(df_all[x].isnull(),1,0)
    print('\nMissing Comparison for -{}\n{}'.format(x,df_all.drop_duplicates(['FIPS','miss_{}'.format(x)]).groupby(['miss_{}'.format(x)])['POP_ESTIMATE_2018'].describe())) | # Models - data transformation


AttributeError: 'str' object has no attribute 'to'

In [43]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./output_dir",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,  # Your training dataset
    eval_dataset=val_data,     # Your validation dataset
)

In [ ]:
trainer.train()

In [54]:
# training loop
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training epoch{epoch+1}/{num_epochs} Batch size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids = inputs, labels = targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
        torch.cuda.empty_cache()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    #validation
    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    valid_iterator = tqdm(val_loader, desc=f"Validating epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(valid_iterator)

    end_time = time.time()  # End the timer for the epoch
    epoch_duration_sec = end_time - start_time  # Calculate the duration in seconds

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}  # Add epoch_duration to the dataframe

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(val_loader)}")

Training epoch1/5 Batch size: 9, Transformer: sagard21/python-code:   0%| | 0/73

import pandas as pd
import os | # Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.
//export
import Path
import TensorFlow
import Python | # A CNN Mnist Model
import seaborn as sns
sns.set(context='talk', style='ticks', font_scale=1.6)
%matplotlib inline | # LSLGA/DR8 QA
model_2.score(Xlin, y) | We can check the $ R^2 $ model score. By linearizing the normalised ENTHRU energy, we have improved our $ R^2 $ score and are thus fitting a model which better describes 

Training epoch1/5 Batch size: 9, Transformer: sagard21/python-code:   0%| | 0/73


OutOfMemoryError: CUDA out of memory. Tried to allocate 576.00 MiB. GPU 0 has a total capacity of 15.81 GiB of which 475.38 MiB is free. Including non-PyTorch memory, this process has 11.73 GiB memory in use. Process 5718 has 3.21 GiB memory in use. Of the allocated memory 10.95 GiB is allocated by PyTorch, and 144.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [57]:
train_iterator

In [56]:
start_time = time.time()
model.train()
epoch_training_loss = 0
train_iterator = tqdm(train_loader, desc=f"Training epoch{epoch+1}/{num_epochs} Batch size: {BATCH_SIZE}, Transformer: {model_name}")
for batch in train_iterator:
    optimizer.zero_grad()
    print(batch)t
    inputs = batch['input_ids'].squeeze(1).to(device)
    break

Training epoch1/5 Batch size: 9, Transformer: sagard21/python-code:   0%| | 0/73

# Plotting the confusion matrix on a heat map

plot_confusion_matrix(confusion_matrix(y_test, y_pred), figsize = (15,12))
plt.ylabel('True Label', fontsize = 20)
plt.xlabel('Predicted Label',  fontsize = 20)
plt.title('Confusion Matrix Heat Map', fontsize = 20)
plt.show() | ### Adding another RNN layer
print ("test: " + test) | ## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. You only need to write code between the ### START CODE HERE ### and ### END CODE HERE ### comments. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run Cell" (denoted by a play symbol) in the upper bar of the notebook. 

We will often specify "(≈ X lines of code)" in the comments to tell you about how much code you need to write. It is just a rough estimate, so don't feel bad if your code is longer or shorter.

**Exercise**: Set test to `"Hello World"` in

In [52]:
# Testing the output of the model
input_str = "def get_string_from_code(node, lines):\n  line_start = node.start_point[0]\n  line_end = node.end_point[0]\n  char_start = node.start_point[1]\n  char_end = node.end_point[1]\n  if line_start != line_end:\n    code_list.append(' '.join([lines[line_start][char_start:]] + lines[line_start+1:line_end] + [lines[line_end][:char_end]]))\n  else:\n    code_list.append(lines[line_start][char_start:char_end])\n\ndef my_traverse(node, code_list):\n  lines = code.split('\n')\n  if node.child_count == 0:\n    get_string_from_code(node, lines)\n  elif node.type == 'string':\n    get_string_from_code(node, lines)\n  else:\n    for n in node.children:\n      my_traverse(n, code_list)\n \n  return ' '.join(code_list)"
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

output = model.generate(
    input_ids,
    max_length=1024,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

get_string_from_code(node, lines):
  """Get string from code.
1. Get start and end point of the node as a list.
2. Add to the code_list a variable called "lines" that is stored in the code object.
3. Iterate through all nodes and traverse them into a dictionary and append their respective lists.
4. For each child node, we get its value from node's start position and char positions by using split() so we can use the Split method on it.



In [ ]:
torch.save(model, "JupyterLM.pt")

In [45]:
model.save_pretrained("model", from_pt=True)

In [ ]:
data = data.map(lambda samples: tokenizer(samples["source_x"]), batched=True)
data

In [ ]:
from transformers import TextStreamer
model.config.use_cache = True
model.eval()